<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/FizzBuzz_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow

In [0]:
func bitEncode(_ n: UInt, bits: Int = 10) -> [Float] {
    var array = [Float](repeating: 0, count: bits)
    for i in 0..<bits {
        array[bits - 1 - i] = Float(n >> i & 1)
    }
    return array
}

func labelEncode(_ n: UInt) -> [Float] {
    guard n > 0 else { return [0, 0, 0, 1] }
    var array: [Float] = [0, 0, 0, 0]

    if n % 15 == 0 {
        array[0] = 1
    }
    else if n % 3 == 0 {
        array[1] = 1
    }
    else if n % 5 == 0 {
        array[2] = 1
    }
    else {
        array[3] = 1
    }

    return array
}

In [0]:
let start = 101
let end = 1024
var X = [[Float]]()
var Y = [[Float]]()

for i in start..<end {
    X.append(bitEncode(UInt(i)))
    Y.append(labelEncode(UInt(i)))
}

In [0]:
struct FizzBuzz: Layer {
    var l1: Dense<Float>
    var l2: Dense<Float>

    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 50, activation: tanh)
        l2 = Dense<Float>(inputSize: 50, outputSize: 4, activation: softmax) // {$0}
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        let output = l1.applied(to: input)
        return l2.applied(to: output)

    }
}

In [5]:
let x = Tensor<Float>(shape: [Int32(end - start), 10], scalars: Array(X.joined()))
let y = Tensor<Float>(shape: [Int32(end - start), 4], scalars: Array(Y.joined()))

print(x.shape)
print(y.shape)

TensorShape(dimensions: [923, 10])
TensorShape(dimensions: [923, 4])


In [16]:
let optimizer = RMSProp<FizzBuzz, Float>(learningRate: 0.03)
var model = FizzBuzz(variables: 10)


for epoch in 1...10000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x)
        //return meanSquaredError(predicted: ŷ, expected: y)
        return softmaxCrossEntropy(logits: ŷ, labels: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 1000 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 1000 Cost: 1.1768423
Epoch: 2000 Cost: 1.1716907
Epoch: 3000 Cost: 1.1489633
Epoch: 4000 Cost: 1.0425111
Epoch: 5000 Cost: 0.8467724
Epoch: 6000 Cost: 0.803761
Epoch: 7000 Cost: 0.8016894
Epoch: 8000 Cost: 0.7999455
Epoch: 9000 Cost: 0.7961145
Epoch: 10000 Cost: 0.79708433
Epoch: 11000 Cost: 0.7959698
Epoch: 12000 Cost: 0.7967748
Epoch: 13000 Cost: 0.7967497
Epoch: 14000 Cost: 0.79602313
Epoch: 15000 Cost: 0.79610425
Epoch: 16000 Cost: 0.796049
Epoch: 17000 Cost: 0.7959788


: ignored

In [0]:
var X_test = [[Float]]()
var Y_true = [[Float]]()

for i in 0..<100 {
    X_test.append(bitEncode(UInt(i)))
    Y_true.append(labelEncode(UInt(i)))
}

let xTest = Tensor<Float>(shape: [100, 10], scalars: Array(X_test.joined()))
let yTest = model.applied(to: xTest)

In [8]:
let yMax = yTest.argmax(squeezingAxis: 1).array
let yArray = Array(yTest.array)
let description = ["FizzBuzz", "Fizz    ", "Buzz    ", "        "]

for i in 0..<100 {
    print(i, description[Int(yMax[i].description)!], Y_true[i], yArray[i])
}

0          [0.0, 0.0, 0.0, 1.0] [0.03104766, 0.22917551, 0.03687466, 0.70290214]
1          [0.0, 0.0, 0.0, 1.0] [0.013110635, 0.22353785, 0.017794633, 0.74555683]
2          [0.0, 0.0, 0.0, 1.0] [0.010598051, 0.20600991, 0.010907119, 0.7724849]
3          [0.0, 1.0, 0.0, 0.0] [0.005083005, 0.19560903, 0.00587185, 0.7934361]
4          [0.0, 0.0, 0.0, 1.0] [0.015403566, 0.14548033, 0.013219786, 0.8258963]
5          [0.0, 0.0, 1.0, 0.0] [0.006811601, 0.14471091, 0.0068619354, 0.84161556]
6          [0.0, 1.0, 0.0, 0.0] [0.005727165, 0.13162926, 0.004264116, 0.8583795]
7          [0.0, 0.0, 0.0, 1.0] [0.0029154378, 0.1304868, 0.002589199, 0.86400855]
8          [0.0, 0.0, 0.0, 1.0] [0.013146905, 0.18292326, 0.021933408, 0.7819964]
9          [0.0, 1.0, 0.0, 0.0] [0.005595836, 0.17396554, 0.010341, 0.81009763]
10          [0.0, 0.0, 1.0, 0.0] [0.0048841033, 0.16894081, 0.0070882635, 0.8190868]
11          [0.0, 0.0, 0.0, 1.0] [0.0023550966, 0.15796624, 0.0038289197, 0.83584976]
12       